In [1]:
# !pip install --upgrade tensorflow
!pip install tensorflow==2.15
# !pip install --upgrade tensorflow
!pip install -U tf_keras # Keras 2
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

!pip install transformers datasets tensorflow

  Using cached tensorflow-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (475.2 MB)
  Using cached ml_dtypes-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached tensorboard-2.15.2-py3-none-any.whl (5.5 MB)
  Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
  Attempting uninstall: keras
    Found existing installation: keras 3.2.1
    Uninstalling keras-3.2.1:
      Successfully uninstalled keras-3.2.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.1
    Uninstalling tensorflow-2.16.1:
      Successfully uninstalled tensorflow-2.16.1
ERROR: pip's dependency re

In [2]:
!pip install datasets
!pip install transformers[torch]
!pip install evaluate
!pip install peft
!pip install accelerate

In [3]:
# !pip install --upgrade transformers

In [4]:
# !pip show tensorflow
# !pip show Keras

In [5]:
import os
import tensorflow as tf
from transformers import TFElectraForSequenceClassification, ElectraTokenizer
from datasets import load_dataset, load_metric
from google.colab import drive

In [6]:
from datasets import load_dataset, load_metric
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
import evaluate
import torch
import numpy as np

In [7]:
torch.backends.cuda.matmul.allow_tf32 = False

In [8]:
drive.mount('/content/drive')

# Checkpoints and final model
checkpoint_dir = '/content/drive/My Drive/W266_Final_Project/final_project/LoRA_Finetune_BASE_GLUE/electra-base-qnli_freezefirst4_6epochs'
final_model_path = '/content/drive/My Drive/W266_Final_Project/final_project/LoRA_Finetune_BASE_GLUE/electra-base-qnli-final_freezefirst4_6epochs'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
ls = [0,1,2,3,4,5,6,7,8,9,10,11]
len(ls), ls[:4], ls[4:]

(12, [0, 1, 2, 3], [4, 5, 6, 7, 8, 9, 10, 11])

In [10]:
# Load ELECTRA BASE model and tokenizer
model_checkpoint = "google/electra-base-discriminator"
# tokenizer = ElectraTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# model = TFElectraForSequenceClassification.from_pretrained(model_name, num_labels=2)

id2label = {
    0: "neg",
    1: "pos"
}

label2id = {
    "neg": 0,
    "pos": 1
}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
for name, param in model.named_parameters():
  if "electra.encoder.layer.0" in name:
    param.requires_grad = False
    print(name, param.requires_grad)

electra.encoder.layer.0.attention.self.query.weight False
electra.encoder.layer.0.attention.self.query.bias False
electra.encoder.layer.0.attention.self.key.weight False
electra.encoder.layer.0.attention.self.key.bias False
electra.encoder.layer.0.attention.self.value.weight False
electra.encoder.layer.0.attention.self.value.bias False
electra.encoder.layer.0.attention.output.dense.weight False
electra.encoder.layer.0.attention.output.dense.bias False
electra.encoder.layer.0.attention.output.LayerNorm.weight False
electra.encoder.layer.0.attention.output.LayerNorm.bias False
electra.encoder.layer.0.intermediate.dense.weight False
electra.encoder.layer.0.intermediate.dense.bias False
electra.encoder.layer.0.output.dense.weight False
electra.encoder.layer.0.output.dense.bias False
electra.encoder.layer.0.output.LayerNorm.weight False
electra.encoder.layer.0.output.LayerNorm.bias False


In [12]:
for name, param in model.named_parameters():
  if "electra.encoder.layer.1" in name:
    if "electra.encoder.layer.10" in name or "electra.encoder.layer.11" in name:
      continue
    param.requires_grad = False
    print(name, param.requires_grad)

electra.encoder.layer.1.attention.self.query.weight False
electra.encoder.layer.1.attention.self.query.bias False
electra.encoder.layer.1.attention.self.key.weight False
electra.encoder.layer.1.attention.self.key.bias False
electra.encoder.layer.1.attention.self.value.weight False
electra.encoder.layer.1.attention.self.value.bias False
electra.encoder.layer.1.attention.output.dense.weight False
electra.encoder.layer.1.attention.output.dense.bias False
electra.encoder.layer.1.attention.output.LayerNorm.weight False
electra.encoder.layer.1.attention.output.LayerNorm.bias False
electra.encoder.layer.1.intermediate.dense.weight False
electra.encoder.layer.1.intermediate.dense.bias False
electra.encoder.layer.1.output.dense.weight False
electra.encoder.layer.1.output.dense.bias False
electra.encoder.layer.1.output.LayerNorm.weight False
electra.encoder.layer.1.output.LayerNorm.bias False


In [13]:
for name, param in model.named_parameters():
  if "electra.encoder.layer.2" in name:
    param.requires_grad = False
    print(name, param.requires_grad)

electra.encoder.layer.2.attention.self.query.weight False
electra.encoder.layer.2.attention.self.query.bias False
electra.encoder.layer.2.attention.self.key.weight False
electra.encoder.layer.2.attention.self.key.bias False
electra.encoder.layer.2.attention.self.value.weight False
electra.encoder.layer.2.attention.self.value.bias False
electra.encoder.layer.2.attention.output.dense.weight False
electra.encoder.layer.2.attention.output.dense.bias False
electra.encoder.layer.2.attention.output.LayerNorm.weight False
electra.encoder.layer.2.attention.output.LayerNorm.bias False
electra.encoder.layer.2.intermediate.dense.weight False
electra.encoder.layer.2.intermediate.dense.bias False
electra.encoder.layer.2.output.dense.weight False
electra.encoder.layer.2.output.dense.bias False
electra.encoder.layer.2.output.LayerNorm.weight False
electra.encoder.layer.2.output.LayerNorm.bias False


In [14]:
for name, param in model.named_parameters():
  if "electra.encoder.layer.3" in name:
    param.requires_grad = False
    print(name, param.requires_grad)

electra.encoder.layer.3.attention.self.query.weight False
electra.encoder.layer.3.attention.self.query.bias False
electra.encoder.layer.3.attention.self.key.weight False
electra.encoder.layer.3.attention.self.key.bias False
electra.encoder.layer.3.attention.self.value.weight False
electra.encoder.layer.3.attention.self.value.bias False
electra.encoder.layer.3.attention.output.dense.weight False
electra.encoder.layer.3.attention.output.dense.bias False
electra.encoder.layer.3.attention.output.LayerNorm.weight False
electra.encoder.layer.3.attention.output.LayerNorm.bias False
electra.encoder.layer.3.intermediate.dense.weight False
electra.encoder.layer.3.intermediate.dense.bias False
electra.encoder.layer.3.output.dense.weight False
electra.encoder.layer.3.output.dense.bias False
electra.encoder.layer.3.output.LayerNorm.weight False
electra.encoder.layer.3.output.LayerNorm.bias False


In [15]:
for name, param in model.named_parameters():
  print(name, param.requires_grad)

electra.embeddings.word_embeddings.weight True
electra.embeddings.position_embeddings.weight True
electra.embeddings.token_type_embeddings.weight True
electra.embeddings.LayerNorm.weight True
electra.embeddings.LayerNorm.bias True
electra.encoder.layer.0.attention.self.query.weight False
electra.encoder.layer.0.attention.self.query.bias False
electra.encoder.layer.0.attention.self.key.weight False
electra.encoder.layer.0.attention.self.key.bias False
electra.encoder.layer.0.attention.self.value.weight False
electra.encoder.layer.0.attention.self.value.bias False
electra.encoder.layer.0.attention.output.dense.weight False
electra.encoder.layer.0.attention.output.dense.bias False
electra.encoder.layer.0.attention.output.LayerNorm.weight False
electra.encoder.layer.0.attention.output.LayerNorm.bias False
electra.encoder.layer.0.intermediate.dense.weight False
electra.encoder.layer.0.intermediate.dense.bias False
electra.encoder.layer.0.output.dense.weight False
electra.encoder.layer.0.out

In [16]:
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [17]:
_= model.to("cuda")

In [18]:
# Load and preprocess the GLUE QNLI dataset
# https://huggingface.co/docs/datasets/v1.1.2/loading_datasets.html
# https://huggingface.co/docs/datasets/en/process
# https://huggingface.co/datasets/nyu-mll/glue
dataset = load_dataset("glue", "qnli")
metric = load_metric("glue", "qnli")

encoded_dataset = dataset.map(lambda examples: tokenizer(examples["question"],
                                                         examples["sentence"],
                                                         truncation=True,
                                                         padding="max_length",
                                                         return_tensors="tf"),
                              batched=True)

<ipython-input-18-86000d1cd3d5>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "qnli")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Map:   0%|          | 0/5463 [00:00<?, ? examples/s]

In [19]:
from datasets import list_metrics
metric = load_metric('accuracy')

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [20]:
lr = 5e-5
batch_size = 32
num_epochs = 6

In [21]:
training_args = TrainingArguments(
    output_dir=checkpoint_dir,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=1,
    load_best_model_at_end=True,
)

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.540800,0.240480,0.909940
2,0.017000,0.227665,0.924217
3,0.009100,0.244239,0.920740
4,0.003000,0.352919,0.926231
5,0.024800,0.420021,0.926780
6,0.000400,0.451814,0.927146


TrainOutput(global_step=19644, training_loss=0.11407391208046072, metrics={'train_runtime': 14029.4453, 'train_samples_per_second': 44.796, 'train_steps_per_second': 1.4, 'total_flos': 1.6535424762943488e+17, 'train_loss': 0.11407391208046072, 'epoch': 6.0})

In [24]:
trainer.evaluate()

{'eval_loss': 0.2276652455329895,
 'eval_accuracy': 0.9242174629324547,
 'eval_runtime': 45.2427,
 'eval_samples_per_second': 120.749,
 'eval_steps_per_second': 3.78,
 'epoch': 6.0}

In [25]:
# Save the final model
trainer.save_model(final_model_path)